# XGBoost Training on Google Cloud Machine Learning Engine

# How to bring your model to ML Engine
Getting your model ready for training can be done in 3 steps:
1. Create your python model file
    1. Add code to download your data from [Google Cloud Storage](https://cloud.google.com/storage) so that ML Engine can use it
    1. Add code to export and save the model to [Google Cloud Storage](https://cloud.google.com/storage) once ML Engine finishes training the model
1. Prepare a package
1. Submit the training job

# Prerequisites
Before you jump in, let’s cover some of the different tools you’ll be using to get online prediction up and running on ML Engine. 

[Google Cloud Platform](https://cloud.google.com/) lets you build and host applications and websites, store data, and analyze data on Google's scalable infrastructure.

[Cloud ML Engine](https://cloud.google.com/ml-engine/) is a managed service that enables you to easily build machine learning models that work on any type of data, of any size.

[Google Cloud Storage](https://cloud.google.com/storage/) (GCS) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

[Cloud SDK](https://cloud.google.com/sdk/) is a command line tool which allows you to interact with Google Cloud products. In order to run this notebook, make sure that Cloud SDK is [installed](https://cloud.google.com/sdk/downloads) in the same environment as your Jupyter kernel.


# Part 0: Setup
* [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
* [Create a Google Cloud Storage Bucket](https://cloud.google.com/storage/docs/quickstart-console)
* [Enable Cloud Machine Learning Engine and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component&_ga=2.217405014.1312742076.1516128282-1417583630.1516128282)
* [Install Cloud SDK](https://cloud.google.com/sdk/downloads)
* [[Optional] Install XGBoost](http://xgboost.readthedocs.io/en/latest/build.html)
* [[Optional] Install scikit-learn](http://scikit-learn.org/stable/install.html)
* [[Optional] Install pandas](https://pandas.pydata.org/pandas-docs/stable/install.html)
* [[Optional] Install Google API Python Client](https://github.com/google/google-api-python-client)

These variables will be needed for the following steps.
* `TRAINER_PACKAGE_PATH <./census_training>` - A packaged training application that will be staged in a Google Cloud Storage location. The model file created below is placed inside this package path.
* `MAIN_TRAINER_MODULE <census_training.train>` - Tells ML Engine which file to execute. This is formatted as follows <folder_name.python_file_name>
* `JOB_DIR <gs://$BUCKET_ID/xgb_job_dir>` - The path to a Google Cloud Storage location to use for job output.
* `RUNTIME_VERSION <1.9>` - The version of Cloud ML Engine to use for the job. If you don't specify a runtime version, the training service uses the default Cloud ML Engine runtime version 1.0. See the list of runtime versions for more information.
* `PYTHON_VERSION <3.5>` - The Python version to use for the job. Python 3.5 is available with runtime version 1.4 or greater. If you don't specify a Python version, the training service uses Python 2.7.

** Replace: **
* `PROJECT_ID <YOUR_PROJECT_ID>` - with your project's id. Use the PROJECT_ID that matches your Google Cloud Platform project.
* `BUCKET_ID <YOUR_BUCKET_ID>` - with the bucket id you created above.
* `JOB_DIR <gs://YOUR_BUCKET_ID/xgb_job_dir>` - with the bucket id you created above.
* `REGION <REGION>` - select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'. The region is where the model will be deployed.

In [1]:
import os

In [2]:
os.listdir('.')

['.catboostModelRegressor.py.swp',
 '.coverage',
 '.coverage.AMOENUS.local.51585.212616',
 '.ipynb_checkpoints',
 '.pytest_cache',
 '__pycache__',
 'catboost_info',
 'feature_importance.png',
 'fork_submission.csv',
 'gcp_training',
 'htmlcov',
 'README.md',
 'submit_gcp_job.ipynb',
 'tests',
 'visualize_processed.ipynb',
 'xgboost.ipynb',
 'xgboostModelRegressor.py']

In [3]:
os.listdir('./gcp_training/')

['.ipynb_checkpoints',
 '.pytest_cache',
 '__init__.py',
 '__pycache__',
 'data',
 'google_cloud_storage_APIs',
 'job_results',
 'README.md',
 'rounder.py',
 'train.py',
 'train_test.py',
 'xgboostModelRegressor.py']

In [6]:
%env PROJECT_ID kaggle-pets
%env BUCKET_ID kaggle-pets-dataset
%env REGION US
%env TRAINER_PACKAGE_PATH ./gcp_training/
%env MAIN_TRAINER_MODULE ./gcp_training/train
%env JOB_DIR gs://kaggle-pets-dataset/
%env RUNTIME_VERSION 1.9
%env PYTHON_VERSION 3.5

env: PROJECT_ID=kaggle-pets
env: BUCKET_ID=kaggle-pets-dataset
env: REGION=US
env: TRAINER_PACKAGE_PATH=./gcp_training/
env: MAIN_TRAINER_MODULE=./gcp_training/train
env: JOB_DIR=gs://kaggle-pets-dataset/
env: RUNTIME_VERSION=1.9
env: PYTHON_VERSION=3.5


# Part 1: Create your python model file

First, we'll create the python model file (provided below) that we'll upload to ML Engine. This is similar to your normal process for creating a XGBoost model. However, there are two key differences:
1. Downloading the data from GCS at the start of your file, so that ML Engine can access the data.
1. Exporting/saving the model to GCS at the end of your file, so that you can use it for predictions.

The code in this file loads the data into a pandas DataFrame and pre-processes the data with scikit-learn. This data is then loaded into a DMatrix and used to train a model. Lastly, the model is saved to a file that can be uploaded to [ML Engine's prediction service](https://cloud.google.com/ml-engine/docs/scikit/getting-predictions#deploy_models_and_versions).



Note: In normal practice you would want to test your model locally on a small dataset to ensure that it works, before using it with your larger dataset on ML Engine. This avoids wasted time and costs.

In [14]:
import os
"python model file is inside "+os.environ['MAIN_TRAINER_MODULE']

'python model file is inside ./gcp_training/train'

In [16]:
"you can test is with "+os.environ['TRAINER_PACKAGE_PATH']+"train_test.py"

'you can test is with ./gcp_training/train_test.py'

# Part 3: Submit Training Job
Next we need to submit the job for training on ML Engine. We'll use gcloud to submit the job which has the following flags:

* `job-name` - A name to use for the job (mixed-case letters, numbers, and underscores only, starting with a letter). In this case: `census_training_$(date +"%Y%m%d_%H%M%S")`
* `job-dir` - The path to a Google Cloud Storage location to use for job output.
* `package-path` - A packaged training application that is staged in a Google Cloud Storage location. If you are using the gcloud command-line tool, this step is largely automated.
* `module-name` - The name of the main module in your trainer package. The main module is the Python file you call to start the application. If you use the gcloud command to submit your job, specify the main module name in the --module-name argument. Refer to Python Packages to figure out the module name.
* `region` - The Google Cloud Compute region where you want your job to run. You should run your training job in the same region as the Cloud Storage bucket that stores your training data. Select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'.
* `runtime-version` - The version of Cloud ML Engine to use for the job. If you don't specify a runtime version, the training service uses the default Cloud ML Engine runtime version 1.0. See the list of runtime versions for more information.
* `python-version` - The Python version to use for the job. Python 3.5 is available with runtime version 1.4 or greater. If you don't specify a Python version, the training service uses Python 2.7.
* `scale-tier` - A scale tier specifying the type of processing cluster to run your job on. This can be the CUSTOM scale tier, in which case you also explicitly specify the number and type of machines to use.

Note: Check to make sure gcloud is set to the current PROJECT_ID

In [17]:
! gcloud config set project kaggle-pets

Updated property [core/project].


Submit the training job.

In [18]:
! gcloud ml-engine jobs submit training xgboost_regressor_$(date +"%Y%m%d_%H%M%S") \
  --job-dir $JOB_DIR \
  --package-path $TRAINER_PACKAGE_PATH \
  --module-name $MAIN_TRAINER_MODULE \
  --region $REGION \
  --runtime-version=$RUNTIME_VERSION \
  --python-version=$PYTHON_VERSION \
  --scale-tier BASIC

API [ml.googleapis.com] not enabled on project [833584618912]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  ^C


Command killed by keyboard interrupt



# [Optional] StackDriver Logging
You can view the logs for your training job:
1. Go to https://console.cloud.google.com/
1. Select "Logging" in left-hand pane
1. Select "Cloud ML Job" resource from the drop-down
1. In filter by prefix, use the value of $JOB_NAME to view the logs

# [Optional] Verify Model File in GCS
View the contents of the destination model folder to verify that model file has indeed been uploaded to GCS.

Note: The model can take a few minutes to train and show up in GCS.

In [ ]:
! gsutil ls gs://$BUCKET_ID/census_*

# Next Steps:
The Cloud Machine Learning Engine online prediction service manages computing resources in the cloud to run your models. Check out the [documentation pages](https://cloud.google.com/ml-engine/docs/scikit/) that describe the process to get online predictions from these exported models using Cloud Machine Learning Engine.